# Comparing different Neural Networks

Each Network uses a vector of the eigen-eigen parts and the eigenvalues and computes the rest-eigen contribution

In [ ]:
using LmaPredict, Flux, Statistics, Plots, Random, StatsBase, JLD2, PrettyTables

## Reading the data

In [ ]:
const path_config = "/Users/lukasgeyer/Studium/Computational Sciences/Masterarbeit/Daten Simon/dat"
const path_plot = "../plots"

In [ ]:
fname = readdir(path_config)[2:5001]
idx = sortperm( parse.(Int64, fname))
fname = fname[idx]
em_n = "VV"

cnfgarr = Vector{LMAConfig}(undef, 0)
for f in fname
    push!(cnfgarr, get_LMAConfig(joinpath(path_config, f), "g5-g5", em=em_n, bc=false))
end

## Splitting data in training and test sets

In [ ]:
NCNFG = length(cnfgarr)
train_size = 500
test_size = NCNFG - train_size

TSRC = "24"
TVALS = length(cnfgarr[1].data["rr"][TSRC]) - 1
if em_n == "PA"
    EIGVALS = 32
else 
    EIGVALS = 64
end

eigvals_data_train = Array{Float64}(undef, EIGVALS, train_size)
rr_data_train = Array{Float64}(undef, TVALS, train_size)
ee_data_train = Array{Float64}(undef, TVALS, train_size)
re_data_train = Array{Float64}(undef, TVALS, train_size)

eigvals_data_test = Array{Float64}(undef, EIGVALS, test_size)
rr_data_test = Array{Float64}(undef, TVALS, test_size)
ee_data_test = Array{Float64}(undef, TVALS, test_size)
re_data_test = Array{Float64}(undef, TVALS, test_size)

for (k, dd) in enumerate(getfield.(cnfgarr, :data)[1:train_size])
    eigvals_data_train[:,k] = copy(cnfgarr[k].data["eigvals"][1:EIGVALS])
    rr_data_train[:,k] = getindex(getindex(dd, "rr"), TSRC)[2:end]
    ee_data_train[:,k] = getindex(getindex(dd, "ee"), TSRC)[2:end]
    re_data_train[:,k] = getindex(getindex(dd, "re"), TSRC)[2:end]
end
for (k, dd) in enumerate(getfield.(cnfgarr, :data)[train_size+1:NCNFG])
    eigvals_data_test[:,k] = copy(cnfgarr[k].data["eigvals"][1:EIGVALS])
    rr_data_test[:,k] = getindex(getindex(dd, "rr"), TSRC)[2:end]
    ee_data_test[:,k] = getindex(getindex(dd, "ee"), TSRC)[2:end]
    re_data_test[:,k] = getindex(getindex(dd, "re"), TSRC)[2:end]
end

## Defining and rescaling input and output data

In [ ]:
input_length = TVALS + EIGVALS
output_length = TVALS

input_shape_train = vcat(1 ./ eigvals_data_train, ee_data_train)
output_shape_train = re_data_train

input_shape_test = vcat(1 ./ eigvals_data_test, ee_data_test)
output_shape_test = re_data_test;

### Input

In [ ]:
max_input_train = maximum(input_shape_train)
min_input_train = minimum(input_shape_train)

mean_input_train = mean(mean.([input_shape_train[:,i] for i in 1:train_size]))
std_input_train = std(mean.([input_shape_train[:,i] for i in 1:train_size]))

input_data_train_normalized = (input_shape_train .- max_input_train) ./ (max_input_train - min_input_train)
input_data_train_standardized = (input_shape_train .- mean_input_train) ./ std_input_train

input_data_test_normalized = (input_shape_test .- max_input_train) ./ (max_input_train - min_input_train)
input_data_test_standardized = (input_shape_test .- mean_input_train) ./ std_input_train;

### Output

In [ ]:
max_output_train = maximum(output_shape_train)
min_output_train = minimum(output_shape_train)

mean_output_train = mean(mean.([output_shape_train[:,i] for i in 1:train_size]))
std_output_train = std(mean.([output_shape_train[:,i] for i in 1:train_size]))

output_data_train_normalized = (output_shape_train .- max_output_train) ./ (max_output_train - min_output_train)
output_data_train_standardized = (output_shape_train .- mean_output_train) ./ std_output_train;

## Describing different Neural Networks

We want to compare different Networks with respect to the parameters:

- How many trainable variables has the Network?
- How long does training take?
- How good is the perfomance - measured by the standard deviation of the difference - with respect to
     - How many configurations have been used for the bias correction

In [ ]:
activation_function = NNlib.tanh

models = [
    Chain(
    Dense(input_length => 1, activation_function),
    Dense(1 => output_length, activation_function),
    ),
    Chain(
    Dense(input_length => 10, activation_function),
    Dense(10 => output_length, activation_function),
    ),
    Chain(
    Dense(input_length => 2*input_length, activation_function),
    Dense(2*input_length => output_length, activation_function),
    ),
    Chain(
    Dense(input_length => 2*input_length, activation_function),
    Dense(2*input_length => 2*input_length, activation_function),
    Dense(2*input_length => output_length, activation_function),
    )
] |> f64;

In [ ]:
using Flux:params

optimizer = Flux.Adam(0.01)
loss_function(x,y) = Flux.mse(x,y)
loss_discription = "MSE"

epochs = 100_000
batch_size = 32

percentages_bc = [0.0, 0.01, 0.02, 0.05, 0.1, 0.12]
n_configs_bc = Int.(4500 .* percentages_bc)

loader = Flux.DataLoader((input_data_train_normalized, output_data_train_normalized), batchsize=batch_size, shuffle=true)

for model in models
    parameters = 0
    layers = params(model)
    for layer in layers
        parameters += length(hcat(layer...))
    end

    output_dir = string("benchmarks/ee+ev/", parameters)
    mkpath(output_dir)
    rm(output_dir * "/results.txt", force=true)

    optim = Flux.setup(optimizer, model)
    
    function training()
        losses = []
        for epoch in 1:epochs
            for (x, y) in loader
                loss, grads = Flux.withgradient(model) do m
                    y_hat = m(x)
                    loss_function(y_hat,y)
                end
                Flux.update!(optim, model, grads[1])
                push!(losses, loss) 
            end
        end

        return losses
    end

    losses = training()
    
    minimum_training = minimum(losses)
    maximum_training = maximum(losses)
    average_training = mean(losses)

    out_of_sample_predictions = (model(input_data_test_normalized) .* (max_output_train - min_output_train)) .+ max_output_train
    out_of_sample_error = [loss_function(out_of_sample_predictions[:,i], output_shape_test[:,i]) for i in 1:test_size]

    c = 3333
    p = scatter(output_shape_test[:,c], label="Actual")
    scatter!(p, out_of_sample_predictions[:,c], label="Prediction", legend=:top)
    xaxis!(p,"t")
    yaxis!(p,"rest-eigen")
    savefig(p,output_dir * "/sample.png")

    minimum_test = minimum(out_of_sample_error)
    maximum_test = maximum(out_of_sample_error)
    average_test = mean(out_of_sample_error)

    open(output_dir * "/results.txt", "a") do file
        println(file, "Time source position: ", TSRC)
        println(file, "Number of used eigenvalues: ", EIGVALS, "\n")

        println(file, "Optimizer: ", optimizer)
        println(file, "Loss function: ", loss_discription)
        println(file, "Epochs: ", epochs)
        println(file, "Batch size: ", batch_size, "\n")
        
        println(file, "Minimum training error (∑⁴⁷ ", loss_discription, "): ", minimum_training)
        println(file, "Maximum training error (∑⁴⁷ ", loss_discription, "): ", maximum_training)
        println(file, "Average training error (∑⁴⁷ ", loss_discription, "): ", average_training, "\n")
        
        println(file, "Minimum test error (∑⁴⁷ ", loss_discription, "): ",  minimum_test)
        println(file, "Maximum test error (∑⁴⁷ ", loss_discription, "): ", maximum_test)
        println(file, "Average test error (∑⁴⁷ ", loss_discription, "): ", average_test, "\n")
    end

    header = [
        "Number of configs used for bc",
        "|μ_diff/μ| (max)",
        "|μ_diff/μ| (min)",
        "|μ_diff/μ| (average)",
    ]

    table = Matrix{Float64}(undef, length(percentages_bc), length(header))

    l = @layout [a b c; d e f]

    means_target = Matrix{Float64}(undef, TVALS, length(percentages_bc))
    stds_target = Matrix{Float64}(undef, TVALS, length(percentages_bc))
    
    means_pred = Matrix{Float64}(undef, TVALS, length(percentages_bc))
    stds_pred = Matrix{Float64}(undef, TVALS, length(percentages_bc))
    
    means_diff = Matrix{Float64}(undef, TVALS, length(percentages_bc))
    stds_diff = Matrix{Float64}(undef, TVALS, length(percentages_bc))
    
    for (i,n) in enumerate(n_configs_bc)
        Random.seed!(10)
        configs = sort!(sample([i for i in 1:test_size], n, replace = false))
        
        uncorr_target_configs = stack(deleteat!([output_shape_test[:,i] for i in 1:test_size],configs), dims=2)

        mean_target = mean.([uncorr_target_configs[i,:] for i in 1:TVALS])
        σ_mean_target = std.([uncorr_target_configs[i,:] for i in 1:TVALS]) ./ sqrt(test_size - 1 - n)
        
        mean_predicted = mean.([out_of_sample_predictions[i,:] for i in 1:TVALS])
        σ_predicted = std.([out_of_sample_predictions[i,:] for i in 1:TVALS])

        if n > 0
            bias_correction = mean(hcat([[out_of_sample_predictions[:,i] - output_shape_test[:,i] for i in configs][i] for i in 1:length(configs)]...), dims=2)
            σ_bc = std(hcat([[out_of_sample_predictions[:,i] - output_shape_test[:,i] for i in configs][i] for i in 1:length(configs)]...), dims=2) ./ sqrt(n - 1)
        else
            bias_correction = zeros(TVALS)
            σ_bc = zeros(TVALS)
        end

        mean_predicted = mean_predicted - bias_correction

        σ_pred_bc = σ_predicted + σ_bc

        mean_diff = (mean_target .- mean_predicted) ./ mean_target
        σ_diff = sqrt.(sum.(hcat([((mean_target .- out_of_sample_predictions[:,i] - bias_correction) ./ mean_target).^2 for i in 1:test_size]...)[k,:] for k in 1:TVALS) ./ (test_size - 1))
        
        means_target[:,i] = mean_target
        stds_target[:,i] = σ_mean_target
        
        means_pred[:,i] = mean_predicted
        stds_pred[:,i] = σ_pred_bc

        means_diff[:,i] = mean_diff
        stds_diff[:,i] = σ_diff

        max_mean_diff = maximum(abs.(mean_diff))
        min_mean_diff = minimum(abs.(mean_diff))
        average_mean_diff = mean(abs.(mean_diff))

        table[i,1] = n
        table[i,2] = max_mean_diff
        table[i,3] = min_mean_diff
        table[i,4] = average_mean_diff
    end

    p = scatter(
        means_diff,
        yerr=stds_diff,
        layout = l,
        size=(1400,1000),
        dpi = 1000,
        legend=:false,
        thickness_scaling = 1.1,
        title=reshape(["bc: $n" for n in n_configs_bc],1,length(n_configs_bc)),
        marker=:+,
        markersize = 2,
        markerstrokewidth = 0.3
    )
    savefig(p,output_dir * "/mean_diff.png")

    p = scatter(
        layout = l,
        size=(1400,1000),
        dpi = 1000,
        thickness_scaling = 1.1,
        title=reshape(["bc: $n" for n in n_configs_bc],1,length(n_configs_bc)))
    
    scatter!(p,
        means_target[7:41,:],
        label="actual",
        legend=:bottom,
        linecolor=:blue,
        marker=:xcross,
        markersize = 2,
        markerstrokewidth = 0.3
    )
    scatter!(p,
        means_pred[7:41,:],
        label="predicted",
        legend=:bottom,
        linecolor=:red,
        marker =:+,
        markersize = 2,
        markerstrokewidth = 0.3
    )
    savefig(p,output_dir * "/mean.png")

    p = scatter(
        layout = l,
        size=(1400,1000),
        dpi = 1000,
        thickness_scaling = 1.1,
        title=reshape(["bc: $n" for n in n_configs_bc],1,length(n_configs_bc)))
    
    scatter!(p,
        means_target[7:41,:],
        yerr=stds_target[7:41,:],
        label="actual",
        legend=:bottom,
        linecolor=:blue,
        marker=:xcross,
        markersize = 2,
        markerstrokewidth = 0.3
    )
    scatter!(p,
        means_pred[7:41,:],
        yerr=stds_pred[7:41,:],
        label="predicted",
        legend=:bottom,
        linecolor=:red,
        marker =:+,
        markersize = 2,
        markerstrokewidth = 0.3
    )
    savefig(p,output_dir * "/mean_errorbar.png")

    open(output_dir * "/results.txt", "a") do file
        pretty_table(file, table, header=header)
        println(file)
        println(file, "Model: ", model)
    end

end